# 1.- Convertimos grabaciones en video (Google Meet) a ficheros de texto

Google Meet ya incorpora esta opción de forma nativa, al menos en el paquete de Google Workspace para uso corporativo. No obstante, disponemos de grabaciones de reuniones pasadas durante años de las que deseamos extraer el conocimiento clave para incoporarlo a nuestro futuro RAG.


In [1]:
!pip install openai-whisper
!pip install pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s 

In [2]:
from IPython import get_ipython
from IPython.display import display

In [3]:
import whisper
import os
from pydub import AudioSegment
from google.cloud import storage
from google.colab import drive

In [6]:
# Configuracion de Whisper
model = whisper.load_model("base")


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 67.9MiB/s]


In [7]:
# Ruta de la carpeta en Google Drive
drive_folder_path = "/content/drive/MyDrive/PF-test" # Ficheros en vídeo, generalmente MP4, de las grabaciones de Google Meet
output_folder_path = "/content/drive/MyDrive/PF-test/transcr" # Carpeta de salida de ficheros de texto


In [9]:
# Iteramos sobre los archivos en la carpeta
for filename in os.listdir(drive_folder_path):
    if filename.endswith(".mp4") or filename.endswith(".mp3") or filename.endswith(".wav"):
        filepath = os.path.join(drive_folder_path, filename)

        # Extraemos el audio del video
        if filename.endswith(".mp4"):
          audio = AudioSegment.from_file(filepath)
          audio.export("temp_audio.wav", format="wav")
          filepath = "temp_audio.wav"

        # Transcripcion con Whisper
        result = model.transcribe(filepath)

        # Guardamos la transcripcion en un archivo de texto en Google Drive con el mismo nombre
        transcription_filename = filename.split(".")[0] + ".txt"
        output_filepath = os.path.join(output_folder_path, transcription_filename)
        with open(output_filepath, "w") as f:
            f.write(result["text"])

        print(f"Transcripción de '{filename}' completa y guardada en Google Drive.")

        # Eliminamos el archivo temporal de audio si existe
        if filename.endswith(".mp4"):
            os.remove("temp_audio.wav")

print("Todas las transcripciones completadas.")

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción de 'Copia de 250313- Clientes VIP.mp4' completa y guardada en Google Drive.


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción de 'Copia de 250227- Stand By.mp4' completa y guardada en Google Drive.


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción de 'Copia de 250313- Formación Aine Villas.mp4' completa y guardada en Google Drive.
Todas las transcripciones completadas.


In [ ]:
#32 minutos para 3 videos. Hacer estimación.

# 2.- Corregir, resumir y mejorar la transcripción.

A continuación intentaremos obtener una mejor versión de estos ficheros con ayuda de GPT4-Turbo para corregir todos los fallos de la transcripción y quedarnos únicamente con lo importante.

In [1]:
!pip install openai==0.28

In [3]:
import openai
import os

from google.cloud import storage
from google.colab import drive
from google.colab import userdata

# Set your OpenAI API key
openai.api_key = userdata.get('OPENAI_API_KEY')  # Stored in Colab notebook Secrets

In [10]:
def summarize_text(text):
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": "Eres un asistente experto en comprensión y resumen de textos."},
            {"role": "user", "content": f"Te paso el texto transcrito, con cierto grado de error, de una reunión de trabajo. Descarta toda la charla cotidiana y trata de corregir el texto teniendo en cuenta que la parte importante versará sobre procedimientos del ámbito inmobiliario y uso de CRM y herramientas de trabajo. Las palabras que no seas capaz de entender por la mala transcripción y consideres que necesitan corrección manual, enuméralas al final y trataremos de mejorar el input:\n\n{text}"}
        ],
        temperature=0.5,
        max_tokens=300
    )

    summary = response['choices'][0]['message']['content'].strip()
    return summary

# Establecemos las rutas de las carpetas de Drive
input_folder_path = "/content/drive/MyDrive/PF-test/transcr"
output_folder_path = "/content/drive/MyDrive/PF-test/summaries"

In [11]:
# Iteramos sobre los archivos .txt en la carpeta de entrada
for filename in os.listdir(input_folder_path):
    if filename.endswith(".txt"):
        input_filepath = os.path.join(input_folder_path, filename)
        output_filepath = os.path.join(output_folder_path, filename)

        try:
            with open(input_filepath, "r") as f:
                text = f.read()

            summary = summarize_text(text)

            with open(output_filepath, "w") as f:
                f.write(summary)

            print(f"Resumen de '{filename}' creado exitosamente.")

        except Exception as e:
            print(f"Error procesando '{filename}': {e}")

print("Todos los resúmenes completados.")

Resumen de 'Copia de 250313- Clientes VIP.txt' creado exitosamente.
Resumen de 'Copia de 250227- Stand By.txt' creado exitosamente.
Resumen de 'Copia de 250313- Formación Aine Villas.txt' creado exitosamente.
Todos los resúmenes completados.


El resultado es bueno teniendo en cuenta la mala calidad del texto de entrada.
Siguientes pasos: mejorar la transcripción y la extracción de conocimiento clave de estos vídeos antes de incorporarlos al RAG.